# Model Improvement for Melanoma Detection

This notebook implements various improvements to our baseline ResNet18 model based on published research and best practices in medical image classification.

## Goals:
1. Implement state-of-the-art architectures (ResNet50, EfficientNet, DenseNet)
2. Apply advanced data augmentation techniques
3. Handle class imbalance with weighted loss and sampling
4. Optimize hyperparameters using Optuna
5. Train and evaluate the best model

## Literature Review & Best Practices

### Key Findings from Research:

**1. Architecture Selection:**
- **EfficientNet** (Tan & Le, 2019): Achieves state-of-the-art accuracy with fewer parameters through compound scaling
  - Source: "EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks" (ICML 2019)
  - Key insight: Balance network depth, width, and resolution for optimal performance

- **ResNet50** (He et al., 2016): Deeper networks with residual connections prevent vanishing gradients
  - Source: "Deep Residual Learning for Image Recognition" (CVPR 2016)
  - Proven effective for medical imaging tasks

- **DenseNet** (Huang et al., 2017): Dense connections improve feature reuse and gradient flow
  - Source: "Densely Connected Convolutional Networks" (CVPR 2017)
  - Efficient parameter usage for medical images

**2. Transfer Learning & Fine-tuning:**
- Pre-training on ImageNet provides robust low-level features (Yosinski et al., 2014)
- Source: "How transferable are features in deep neural networks?" (NIPS 2014)
- Strategy: Fine-tune all layers with lower learning rate for better adaptation

**3. Data Augmentation for Medical Images:**
- **Horizontal/Vertical Flips**: Lesions can appear in any orientation
- **Rotation (±20°)**: Natural variation in image capture angles
- **Color Jittering**: Handles lighting variations in dermatoscopy
- **RandAugment** (Cubuk et al., 2020): Automated augmentation strategy
  - Source: "RandAugment: Practical automated data augmentation" (NeurIPS 2020)
- **MixUp** (Zhang et al., 2018): Reduces overfitting and improves calibration
  - Source: "mixup: Beyond Empirical Risk Minimization" (ICLR 2018)

**4. Class Imbalance Handling:**
- **Weighted Loss**: Penalize misclassification of minority classes more heavily
- **Focal Loss** (Lin et al., 2017): Focus on hard examples
  - Source: "Focal Loss for Dense Object Detection" (ICCV 2017)
- **Oversampling**: Balance training distribution with WeightedRandomSampler

**5. Melanoma-Specific Insights:**
- Input resolution: 224x224 or 299x299 recommended (Esteva et al., 2017)
- Source: "Dermatologist-level classification of skin cancer with deep neural networks" (Nature 2017)
- Multi-scale features important for detecting subtle patterns
- Ensemble methods can improve robustness (Haenssle et al., 2018)

**6. Hyperparameter Optimization:**
- Learning rate: Critical for convergence (typically 1e-4 to 1e-3 for fine-tuning)
- Batch size: Trade-off between generalization and training speed
- Weight decay: Regularization to prevent overfitting (typically 1e-5 to 1e-4)
- Optimizer: Adam or AdamW with cosine annealing scheduler

In [1]:
# Install required packages
# !pip install optuna timm albumentations tensorboard scikit-learn

### Fix for NumPy compatibility error

If you get the error: `ValueError: numpy.dtype size changed, may indicate binary incompatibility`

This happens when packages were compiled against different NumPy versions. Run this in your terminal from the project root:

```powershell
.\.venv\Scripts\python.exe -m pip install --upgrade pip
.\.venv\Scripts\python.exe -m pip install --upgrade numpy
.\.venv\Scripts\python.exe -m pip install --force-reinstall --no-cache-dir scipy pandas matplotlib pillow scikit-learn
.\.venv\Scripts\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
.\.venv\Scripts\python.exe -m pip install optuna timm tensorboard seaborn tqdm
```

Or run this in a notebook cell with `!`:
```python
!.\.venv\Scripts\python.exe -m pip install --upgrade pip numpy
!.\.venv\Scripts\python.exe -m pip install --force-reinstall --no-cache-dir scipy pandas matplotlib pillow scikit-learn
!.\.venv\Scripts\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!.\.venv\Scripts\python.exe -m pip install optuna timm tensorboard seaborn tqdm
```

**Important:** After installing, restart the kernel for changes to take effect.

In [2]:
import os
import json
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models

import timm  # PyTorch Image Models for EfficientNet
import optuna
from optuna.trial import TrialState

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

c:\Users\major\Documents\University\DeepLearning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


## 1. Load Existing Model for Comparison

In [3]:
# Load the baseline ResNet18 model
baseline_model = models.resnet18(pretrained=False)
num_ftrs = baseline_model.fc.in_features
baseline_model.fc = nn.Linear(num_ftrs, 3)

if os.path.exists("melanoma_model_weights.pth"):
    baseline_model.load_state_dict(torch.load("melanoma_model_weights.pth", map_location=device))
    print("✓ Baseline ResNet18 model loaded successfully")
else:
    print("⚠ Warning: melanoma_model_weights.pth not found. Will train from scratch.")

baseline_model = baseline_model.to(device)
baseline_model.eval()

c:\Users\major\Documents\University\DeepLearning\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\major\Documents\University\DeepLearning\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✓ Baseline ResNet18 model loaded successfully


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 2. Data Pipeline with Advanced Augmentation

In [4]:
class HAM10000Dataset(Dataset):
    """Dataset class for HAM10000 with flexible augmentation."""

    def __init__(self, image_dir, ann_dir, image_files, transform=None):
        self.image_dir = image_dir
        self.ann_dir = ann_dir
        self.image_files = image_files
        self.transform = transform

        # Class mapping from original labels to 3-class groups
        self.group_map = {
            "melanoma": "melanoma",
            "basal cell carcinoma": "suspicious",
            "actinic keratoses": "suspicious",
            "melanocytic nevi": "benign",
            "benign keratosis-like lesions": "benign",
            "dermatofibroma": "benign",
            "vascular lesions": "benign",
        }

        self.group_to_idx = {"benign": 0, "suspicious": 1, "melanoma": 2}
        self.idx_to_group = {0: "benign", 1: "suspicious", 2: "melanoma"}

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Load annotation
        ann_path = os.path.join(self.ann_dir, img_name + ".json")
        try:
            with open(ann_path, "r") as f:
                ann = json.load(f)
            original_label = ann["objects"][0]["classTitle"]
            group_label = self.group_map[original_label]
            label = self.group_to_idx[group_label]
        except Exception as e:
            # Skip corrupted annotations
            print(f"Error loading {ann_path}: {e}")
            label = 0  # Default to benign

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
def get_augmentation_transforms(augmentation_level="medium", input_size=224):
    """
    Create augmentation transforms based on research best practices.

    Args:
        augmentation_level: 'light', 'medium', or 'heavy'
        input_size: Target image size (224 or 299)
    """

    # Base transforms for validation/testing
    val_transform = transforms.Compose(
        [
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )

    # Training transforms with varying augmentation levels
    if augmentation_level == "light":
        train_transform = transforms.Compose(
            [
                transforms.Resize((input_size, input_size)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    elif augmentation_level == "medium":
        train_transform = transforms.Compose(
            [
                transforms.Resize((input_size, input_size)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(20),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    else:  # heavy
        train_transform = transforms.Compose(
            [
                transforms.Resize((int(input_size * 1.1), int(input_size * 1.1))),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(30),
                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.15),
                transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    return train_transform, val_transform

In [ ]:
# Prepare dataset splits
image_dir = "data/ham10000/ds/img"
ann_dir = "data/ham10000/ds/ann"

# Get all image files
all_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
print(f"Total images found: {len(all_files)}")

# Split into train/val/test (70/15/15)
train_files, test_files = train_test_split(all_files, test_size=0.3, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.3, random_state=42)  # 0.176 * 0.85 ≈ 0.15

print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

Total images found: 10015
Train: 6409, Val: 1603, Test: 2003


In [7]:
# Analyze class distribution
def get_class_distribution(image_files, ann_dir):
    """Get class distribution for given image files."""
    group_map = {
        "melanoma": "melanoma",
        "basal cell carcinoma": "suspicious",
        "actinic keratoses": "suspicious",
        "melanocytic nevi": "benign",
        "benign keratosis-like lesions": "benign",
        "dermatofibroma": "benign",
        "vascular lesions": "benign",
    }

    labels = []
    for img_name in image_files:
        ann_path = os.path.join(ann_dir, img_name + ".json")
        try:
            with open(ann_path, "r") as f:
                ann = json.load(f)
            original_label = ann["objects"][0]["classTitle"]
            group_label = group_map[original_label]
            labels.append(group_label)
        except:
            pass

    return Counter(labels)


train_dist = get_class_distribution(train_files, ann_dir)
print("\nTraining set class distribution:")
for label, count in train_dist.items():
    print(f"  {label}: {count} ({count/sum(train_dist.values())*100:.1f}%)")


Training set class distribution:
  benign: 5152 (80.4%)
  suspicious: 545 (8.5%)
  melanoma: 711 (11.1%)


## 3. Improved Model Architectures

In [8]:
def create_model(model_name="resnet50", num_classes=3, pretrained=True):
    """
    Create a model with transfer learning.

    Supported architectures based on literature:
    - resnet18, resnet50: Deep residual learning (He et al., 2016)
    - efficientnet_b0, efficientnet_b3: Compound scaling (Tan & Le, 2019)
    - densenet121: Dense connections (Huang et al., 2017)
    """

    if model_name.startswith("resnet"):
        if model_name == "resnet18":
            model = models.resnet18(pretrained=pretrained)
        elif model_name == "resnet50":
            model = models.resnet50(pretrained=pretrained)
        else:
            raise ValueError(f"Unsupported ResNet variant: {model_name}")

        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

    elif model_name.startswith("efficientnet"):
        # Using timm library for EfficientNet
        model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)

    elif model_name.startswith("densenet"):
        if model_name == "densenet121":
            model = models.densenet121(pretrained=pretrained)
        else:
            raise ValueError(f"Unsupported DenseNet variant: {model_name}")

        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)

    else:
        raise ValueError(f"Unsupported model: {model_name}")

    return model

## 4. Training Functions with Class Imbalance Handling

In [9]:
def get_class_weights(dataset):
    """
    Calculate class weights for weighted loss.
    Inverse frequency weighting to handle class imbalance.
    """
    labels = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        labels.append(label)

    class_counts = Counter(labels)
    total = sum(class_counts.values())

    # Inverse frequency weighting
    weights = {cls: total / count for cls, count in class_counts.items()}

    # Normalize weights
    weight_sum = sum(weights.values())
    weights = {cls: w / weight_sum * len(weights) for cls, w in weights.items()}

    # Convert to tensor
    weight_tensor = torch.tensor([weights[i] for i in range(len(weights))], dtype=torch.float32)

    return weight_tensor

In [10]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def validate(model, dataloader, criterion, device):
    """Validate the model."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [11]:
def train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, early_stopping_patience=5
):
    """
    Full training loop with early stopping and best model checkpointing.
    """
    best_val_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0

    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    for epoch in range(num_epochs):
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)

        # Validate
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        # Learning rate scheduling
        if scheduler is not None:
            scheduler.step()

        # Record history
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        # Check for best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            patience_counter = 0
            print(f"  ✓ New best validation accuracy: {best_val_acc:.4f}")
        else:
            patience_counter += 1

        # Early stopping
        if patience_counter >= early_stopping_patience:
            print(f"\nEarly stopping triggered after {epoch+1} epochs")
            break

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, history, best_val_acc

## 5. Hyperparameter Optimization with Optuna

In [ ]:
def objective(trial):
    """
    Optuna objective function for hyperparameter optimization.

    PRIMARY METRIC: Melanoma Recall (Sensitivity)
    - Optimizes for catching all melanoma cases (minimize false negatives)
    - More clinically appropriate than overall accuracy
    - Secondary consideration: balanced performance across all classes

    Optimizes:
    - Model architecture
    - Learning rate
    - Weight decay
    - Batch size
    - Augmentation level
    """

    # Suggest hyperparameters
    model_name = trial.suggest_categorical("model", ["resnet18", "resnet50", "efficientnet_b0", "densenet121"])
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    augmentation_level = trial.suggest_categorical("augmentation", ["light", "medium", "heavy"])

    # Determine input size based on model
    input_size = 299 if "efficientnet" in model_name else 224

    # Create datasets with selected augmentation
    train_transform, val_transform = get_augmentation_transforms(augmentation_level, input_size)

    train_dataset = HAM10000Dataset(image_dir, ann_dir, train_files, transform=train_transform)
    val_dataset = HAM10000Dataset(image_dir, ann_dir, val_files, transform=val_transform)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Create model
    model = create_model(model_name, num_classes=3, pretrained=True)
    model = model.to(device)

    # Calculate class weights
    class_weights = get_class_weights(train_dataset).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # Optimizer and scheduler
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

    # Train for limited epochs (hyperparameter search)
    num_epochs = 10

    try:
        model, history, best_val_acc = train_model(
            model,
            train_loader,
            val_loader,
            criterion,
            optimizer,
            scheduler,
            num_epochs=num_epochs,
            device=device,
            early_stopping_patience=3,
        )

        # Evaluate on validation set to get melanoma recall
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels_batch in val_loader:
                images = images.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels_batch.numpy())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)

        # Calculate melanoma recall (class 2)
        from sklearn.metrics import recall_score

        melanoma_preds_binary = (all_preds == 2).astype(int)
        melanoma_labels_binary = (all_labels == 2).astype(int)
        melanoma_recall = recall_score(melanoma_labels_binary, melanoma_preds_binary, zero_division=0)

        # Calculate macro recall for balance
        macro_recall = recall_score(all_labels, all_preds, average="macro", zero_division=0)

        # Composite score: 70% melanoma recall + 30% macro recall
        # This ensures we catch melanoma while maintaining reasonable overall performance
        composite_score = 0.7 * melanoma_recall + 0.3 * macro_recall

        print(
            f"  → Melanoma Recall: {melanoma_recall:.4f}, Macro Recall: {macro_recall:.4f}, Composite: {composite_score:.4f}"
        )

        return composite_score

    except Exception as e:
        print(f"Trial failed with error: {e}")
        return 0.0

In [ ]:
# Run hyperparameter optimization
# Note: This will take significant time. Adjust n_trials based on your compute resources.

study = optuna.create_study(direction="maximize", study_name="melanoma_detection")

# Run optimization with fewer trials for demonstration
# Increase n_trials for more thorough search (recommended: 20-50 trials)
n_trials = 3

print(f"Starting hyperparameter optimization with {n_trials} trials...")
print("PRIMARY METRIC: Melanoma Recall (70%) + Macro Recall (30%)")
print("This may take several hours depending on your hardware.\n")

study.optimize(objective, n_trials=n_trials, timeout=None)

print("\n" + "=" * 70)
print("Hyperparameter Optimization Complete!")
print("=" * 70)
print(f"\nBest trial: {study.best_trial.number}")
print(f"Best composite score (70% melanoma recall + 30% macro recall): {study.best_trial.value:.4f}")
print(f"\nBest hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

print("\n💡 NOTE: This model is optimized for melanoma detection (recall-focused)")
print("   rather than overall accuracy. This is clinically appropriate.")

[I 2025-11-18 18:40:20,728] A new study created in memory with name: melanoma_detection
c:\Users\major\Documents\University\DeepLearning\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\major\Documents\University\DeepLearning\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Starting hyperparameter optimization with 3 trials...
This may take several hours depending on your hardware.

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\major/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:01<00:00, 16.6MB/s]



Error loading data/ham10000/ds/ann\ISIC_0029819.jpg.json: list index out of range


In [ ]:
# Save detailed trial report to text file
from datetime import datetime

report_path = "optuna_trials_report.txt"

with open(report_path, "w") as f:
    # Header
    f.write("=" * 80 + "\n")
    f.write("HYPERPARAMETER OPTIMIZATION REPORT\n")
    f.write("=" * 80 + "\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Study Name: {study.study_name}\n")
    f.write(f"Direction: Maximize (Composite Score = 70% Melanoma Recall + 30% Macro Recall)\n")
    f.write(f"Total Trials: {len(study.trials)}\n")
    f.write("=" * 80 + "\n\n")

    # Best trial summary
    f.write("BEST TRIAL SUMMARY\n")
    f.write("-" * 80 + "\n")
    f.write(f"Trial Number: {study.best_trial.number}\n")
    f.write(f"Best Composite Score: {study.best_trial.value:.6f}\n")
    f.write(f"\nBest Hyperparameters:\n")
    for key, value in study.best_trial.params.items():
        f.write(f"  {key:20s}: {value}\n")
    f.write("\n" + "=" * 80 + "\n\n")

    # Detailed trial results
    f.write("DETAILED TRIAL RESULTS\n")
    f.write("=" * 80 + "\n\n")

    for trial in study.trials:
        f.write(f"Trial #{trial.number}\n")
        f.write("-" * 80 + "\n")
        f.write(f"Status: {trial.state.name}\n")

        if trial.value is not None:
            f.write(f"Composite Score: {trial.value:.6f}\n")
        else:
            f.write(f"Composite Score: FAILED\n")

        f.write(f"\nHyperparameters:\n")
        for key, value in trial.params.items():
            f.write(f"  {key:20s}: {value}\n")

        # Duration
        if trial.duration is not None:
            duration_min = trial.duration.total_seconds() / 60
            f.write(f"\nDuration: {duration_min:.2f} minutes\n")

        # User attributes (if any custom metrics were stored)
        if trial.user_attrs:
            f.write(f"\nAdditional Metrics:\n")
            for key, value in trial.user_attrs.items():
                f.write(f"  {key}: {value}\n")

        f.write("\n" + "=" * 80 + "\n\n")

    # Summary statistics
    f.write("SUMMARY STATISTICS\n")
    f.write("=" * 80 + "\n")

    completed_trials = [t for t in study.trials if t.value is not None]
    if completed_trials:
        scores = [t.value for t in completed_trials]
        f.write(f"Completed Trials: {len(completed_trials)}/{len(study.trials)}\n")
        f.write(f"Best Score: {max(scores):.6f}\n")
        f.write(f"Worst Score: {min(scores):.6f}\n")
        f.write(f"Mean Score: {np.mean(scores):.6f}\n")
        f.write(f"Std Dev: {np.std(scores):.6f}\n")

    f.write("\n" + "=" * 80 + "\n")
    f.write("END OF REPORT\n")
    f.write("=" * 80 + "\n")

print(f"✓ Detailed trial report saved to: {report_path}")
print(f"  Total trials documented: {len(study.trials)}")
print(f"  Best trial: #{study.best_trial.number} with score {study.best_trial.value:.6f}")

In [ ]:
# Visualize optimization history
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

fig = optuna.visualization.plot_param_importances(study)
fig.show()

## 6. Train Final Model with Best Hyperparameters

In [ ]:
# Extract best hyperparameters
best_params = study.best_trial.params
print("Training final model with best hyperparameters:")
print(best_params)

# Setup with best parameters
model_name = best_params["model"]
lr = best_params["lr"]
weight_decay = best_params["weight_decay"]
batch_size = best_params["batch_size"]
augmentation_level = best_params["augmentation"]

input_size = 299 if "efficientnet" in model_name else 224

# Create datasets
train_transform, val_transform = get_augmentation_transforms(augmentation_level, input_size)

train_dataset = HAM10000Dataset(image_dir, ann_dir, train_files, transform=train_transform)
val_dataset = HAM10000Dataset(image_dir, ann_dir, val_files, transform=val_transform)
test_dataset = HAM10000Dataset(image_dir, ann_dir, test_files, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Create model
final_model = create_model(model_name, num_classes=3, pretrained=True)
final_model = final_model.to(device)

# Setup training
class_weights = get_class_weights(train_dataset).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(final_model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

# Train with more epochs for final model
print("\nTraining final model...")
final_model, history, best_val_acc = train_model(
    final_model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler,
    num_epochs=30,
    device=device,
    early_stopping_patience=7,
)

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
ax1.plot(history["train_loss"], label="Train Loss")
ax1.plot(history["val_loss"], label="Val Loss")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.set_title("Training and Validation Loss")
ax1.legend()
ax1.grid(True)

# Accuracy plot
ax2.plot(history["train_acc"], label="Train Acc")
ax2.plot(history["val_acc"], label="Val Acc")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy")
ax2.set_title("Training and Validation Accuracy")
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig("training_history.png", dpi=300, bbox_inches="tight")
plt.show()

## 7. Evaluate on Test Set

### Clinical Metrics: Why Recall Matters

For melanoma detection, **Recall (Sensitivity)** is the most critical metric:

**🔴 High Recall for Melanoma is CRITICAL**:
- **False Negative** (missing melanoma) = Patient doesn't get treatment → Life-threatening
- **False Positive** (incorrectly flagging benign as melanoma) = Unnecessary doctor visit → Inconvenient but safe

**Clinical Priority**:
1. **Melanoma Recall** → Maximize (catch all dangerous cases, even with false positives)
2. **Suspicious Recall** → High (better safe than sorry)
3. **Benign Precision** → Moderate (avoid unnecessary anxiety, but safety first)

**Trade-off**: We prefer 95% melanoma recall with 80% accuracy over 90% accuracy with 70% melanoma recall.

This is why we'll use **weighted recall** or **melanoma-specific recall** as our optimization metric instead of overall accuracy.

In [ ]:
def evaluate_model(model, test_loader, device):
    """Comprehensive model evaluation with metrics."""
    model.eval()

    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())

    return np.array(all_preds), np.array(all_labels), np.array(all_probs)


# Evaluate
preds, labels, probs = evaluate_model(final_model, test_loader, device)

# Calculate metrics
test_acc = accuracy_score(labels, preds)
print(f"\nTest Accuracy: {test_acc:.4f}")

# Classification report
class_names = ["benign", "suspicious", "melanoma"]
print("\nClassification Report:")
print(classification_report(labels, preds, target_names=class_names, digits=4))

# Confusion matrix
cm = confusion_matrix(labels, preds)
print("\nConfusion Matrix:")
print(cm)

In [ ]:
# Calculate detailed clinical metrics with emphasis on Recall
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score

# Per-class metrics
print("\n" + "=" * 70)
print("CLINICAL METRICS - RECALL FOCUSED")
print("=" * 70)

for i, class_name in enumerate(class_names):
    class_preds_binary = (preds == i).astype(int)
    class_labels_binary = (labels == i).astype(int)

    recall = recall_score(class_labels_binary, class_preds_binary)
    precision = precision_score(class_labels_binary, class_preds_binary, zero_division=0)
    f1 = f1_score(class_labels_binary, class_preds_binary, zero_division=0)

    print(f"\n{class_name.upper()}:")
    print(f"  Recall (Sensitivity): {recall:.4f} ⭐ {'CRITICAL!' if class_name == 'melanoma' else ''}")
    print(f"  Precision:            {precision:.4f}")
    print(f"  F1-Score:             {f1:.4f}")
    print(f"  Support:              {np.sum(labels == i)}")

# Calculate macro and weighted recall
macro_recall = recall_score(labels, preds, average="macro")
weighted_recall = recall_score(labels, preds, average="weighted")
melanoma_recall = recall_score((labels == 2).astype(int), (preds == 2).astype(int))

print("\n" + "=" * 70)
print(f"Overall Accuracy:         {test_acc:.4f}")
print(f"Macro Recall (avg):       {macro_recall:.4f}")
print(f"Weighted Recall:          {weighted_recall:.4f}")
print(f"🔴 MELANOMA RECALL:        {melanoma_recall:.4f} ← PRIMARY METRIC")
print("=" * 70)

# Clinical interpretation
if melanoma_recall >= 0.95:
    print("\n✅ EXCELLENT: Melanoma recall ≥95% - Clinically acceptable")
elif melanoma_recall >= 0.90:
    print("\n✓ GOOD: Melanoma recall ≥90% - Acceptable with monitoring")
elif melanoma_recall >= 0.85:
    print("\n⚠ WARNING: Melanoma recall <90% - Needs improvement")
else:
    print("\n❌ CRITICAL: Melanoma recall <85% - NOT clinically safe")

In [ ]:
# Visualize confusion matrix
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.title("Confusion Matrix - Improved Model")
plt.savefig("confusion_matrix_improved.png", dpi=300, bbox_inches="tight")
plt.show()

## 8. Save Improved Model

In [ ]:
# Save model weights
model_save_path = "melanoma_model_improved_weights.pth"
torch.save(final_model.state_dict(), model_save_path)
print(f"✓ Model weights saved to {model_save_path}")

# Save model configuration and results
results = {
    "model_architecture": model_name,
    "input_size": input_size,
    "num_classes": 3,
    "best_hyperparameters": best_params,
    "best_val_accuracy": float(best_val_acc),
    "test_accuracy": float(test_acc),
    "class_names": class_names,
    "training_history": {
        "train_loss": [float(x) for x in history["train_loss"]],
        "val_loss": [float(x) for x in history["val_loss"]],
        "train_acc": [float(x) for x in history["train_acc"]],
        "val_acc": [float(x) for x in history["val_acc"]],
    },
}

import json

with open("model_results_improved.json", "w") as f:
    json.dump(results, f, indent=2)

print("✓ Model configuration and results saved to model_results_improved.json")

## 9. Compare with Baseline Model

## Experimental Results & Model Comparison

### Models Evaluated

We systematically evaluated multiple state-of-the-art architectures through hyperparameter optimization:

#### 1. **ResNet18** (Baseline)
- **Architecture**: He et al., 2016 - "Deep Residual Learning for Image Recognition"
- **Why chosen**: Proven baseline for medical imaging, good balance of performance and speed
- **Parameters**: ~11M
- **Key features**: 18 layers with residual connections
- **Expected use**: Fast inference, baseline comparison

#### 2. **ResNet50**
- **Architecture**: He et al., 2016 - Deeper variant of ResNet
- **Why chosen**: More capacity for complex pattern recognition in skin lesions
- **Parameters**: ~25M
- **Key features**: 50 layers, bottleneck architecture
- **Expected use**: Improved accuracy over ResNet18 with moderate computational cost

#### 3. **EfficientNet-B0**
- **Architecture**: Tan & Le, 2019 - "EfficientNet: Rethinking Model Scaling for CNNs"
- **Why chosen**: State-of-the-art efficiency, compound scaling of depth/width/resolution
- **Parameters**: ~5M
- **Key features**: Optimized architecture through NAS, excellent parameter efficiency
- **Expected use**: Best accuracy-to-efficiency ratio

#### 4. **DenseNet121**
- **Architecture**: Huang et al., 2017 - "Densely Connected Convolutional Networks"
- **Why chosen**: Dense connections promote feature reuse, good for medical imaging
- **Parameters**: ~8M
- **Key features**: Each layer receives gradients from all previous layers
- **Expected use**: Strong gradient flow, efficient feature learning

---

### Hyperparameter Optimization Results

**Optimization Strategy**: Optuna Tree-structured Parzen Estimator (TPE)

**Search Space**:
- **Models**: [ResNet18, ResNet50, EfficientNet-B0, DenseNet121]
- **Learning Rate**: [1e-5, 1e-3] (log scale)
- **Weight Decay**: [1e-6, 1e-3] (log scale)
- **Batch Size**: [16, 32, 64]
- **Augmentation**: [light, medium, heavy]

**Number of Trials**: _[Fill in after running]_

**Best Hyperparameters Found**:
```
Model: [To be filled after optimization]
Learning Rate: [To be filled]
Weight Decay: [To be filled]
Batch Size: [To be filled]
Augmentation Level: [To be filled]
```

---

### Performance Summary

| Model | Val Accuracy | Test Accuracy | Params | Training Time | Notes |
|-------|-------------|---------------|--------|---------------|-------|
| ResNet18 (baseline) | _TBD_ | _TBD_ | 11M | ~X min | Baseline model |
| ResNet50 | _TBD_ | _TBD_ | 25M | ~X min | _Add notes after training_ |
| EfficientNet-B0 | _TBD_ | _TBD_ | 5M | ~X min | _Add notes after training_ |
| DenseNet121 | _TBD_ | _TBD_ | 8M | ~X min | _Add notes after training_ |
| **Best Model** | **_TBD_** | **_TBD_** | **_TBD_** | **~X min** | **Selected via Optuna** |

---

### Per-Class Performance (Best Model)

| Class | Precision | Recall | F1-Score | Support |
|-------|-----------|--------|----------|---------|
| Benign | _TBD_ | _TBD_ | _TBD_ | _TBD_ |
| Suspicious | _TBD_ | _TBD_ | _TBD_ | _TBD_ |
| Melanoma | _TBD_ | _TBD_ | _TBD_ | _TBD_ |

**Key Observations**:
- _[Add observations about which classes are hardest to predict]_
- _[Note any confusion patterns between classes]_
- _[Comment on clinical implications]_

---

### Key Insights from Optimization

**Most Important Hyperparameters** (from Optuna importance analysis):
1. _[To be filled after running plot_param_importances]_
2. _[...]_
3. _[...]_

**Findings**:
- **Data Augmentation**: _[Effect of augmentation level on performance]_
- **Model Capacity**: _[Trade-off between model size and accuracy]_
- **Learning Rate**: _[Optimal range found]_
- **Batch Size**: _[Impact on convergence and generalization]_

**Clinical Relevance**:
- **Sensitivity for Melanoma**: _[Critical metric for dangerous cases]_
- **Specificity for Benign**: _[Avoid unnecessary patient anxiety]_
- **Suspicious Cases**: _[Balance between caution and over-referral]_

---

### Recommendations from Results

Based on experimental results:

1. **Production Model**: _[Best model recommendation]_
2. **Deployment Trade-offs**: 
   - For **mobile/edge devices**: EfficientNet-B0 (smaller, faster)
   - For **cloud/server**: ResNet50 or best Optuna model (highest accuracy)
3. **Ensemble Strategy**: Consider averaging top 3 models for 1-3% accuracy boost
4. **Threshold Tuning**: Adjust decision thresholds based on class-specific requirements

---

**Last Updated**: _[Add date after completing experiments]_

In [ ]:
# Evaluate baseline model on same test set
if os.path.exists("melanoma_model_weights.pth"):
    print("Evaluating baseline ResNet18 model...")
    baseline_preds, baseline_labels, _ = evaluate_model(baseline_model, test_loader, device)
    baseline_acc = accuracy_score(baseline_labels, baseline_preds)

    print("\n" + "=" * 70)
    print("MODEL COMPARISON")
    print("=" * 70)
    print(f"Baseline ResNet18 Test Accuracy: {baseline_acc:.4f}")
    print(f"Improved {model_name} Test Accuracy: {test_acc:.4f}")
    print(f"Improvement: {(test_acc - baseline_acc):.4f} ({(test_acc - baseline_acc)/baseline_acc*100:.2f}%)")
    print("=" * 70)
else:
    print("⚠ Baseline model not found for comparison")

## 10. Next Steps and Recommendations

### Further Improvements:

1. **Ensemble Methods**:
   - Train multiple models (ResNet50, EfficientNet, DenseNet)
   - Average predictions for improved robustness
   - Research shows 2-5% accuracy gains (Haenssle et al., 2018)

2. **Advanced Augmentation**:
   - Implement MixUp (Zhang et al., 2018)
   - Try CutMix (Yun et al., 2019)
   - Test AutoAugment strategies

3. **Test-Time Augmentation (TTA)**:
   - Apply augmentations during inference
   - Average predictions across augmented versions
   - Typically improves accuracy by 1-3%

4. **Focal Loss Implementation**:
   - Replace CrossEntropyLoss with Focal Loss
   - Better handles hard examples
   - Particularly useful for medical imaging

5. **Attention Mechanisms**:
   - Add attention modules (CBAM, SE-Net)
   - Helps model focus on lesion regions
   - Improves interpretability

6. **Multi-Task Learning**:
   - Predict both 3-class groups and original 7 classes
   - Auxiliary tasks can improve feature learning

7. **External Validation**:
   - Test on ISIC2020 dataset
   - Evaluate generalization to different data distributions

### Deployment Considerations:

- **Model Compression**: Use pruning or quantization for mobile deployment
- **ONNX Export**: Convert to ONNX for cross-platform inference
- **API Development**: Create REST API with FastAPI or Flask
- **Monitoring**: Track prediction distributions in production
- **Regulatory**: Consider FDA/CE marking requirements for medical devices

In [ ]:
# Summary statistics
print("\n" + "=" * 70)
print("TRAINING COMPLETE - SUMMARY")
print("=" * 70)
print(f"\nBest Model: {model_name}")
print(f"Input Size: {input_size}x{input_size}")
print(f"Augmentation: {augmentation_level}")
print(f"Learning Rate: {lr:.6f}")
print(f"Weight Decay: {weight_decay:.6f}")
print(f"Batch Size: {batch_size}")
print(f"\nFinal Test Accuracy: {test_acc:.4f}")
print(f"\nModel saved to: {model_save_path}")
print("=" * 70)